<a href="https://colab.research.google.com/github/Chubbychie/faker/blob/master/thonburian_whisper_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/thonburian-whisper/blob/main/thonburian_whisper_notebook.ipynb)


# **Thonburian Whisper**

Automatic Speech Recognition (ASR) model for Thai

<img src="https://raw.githubusercontent.com/biodatlab/thonburian-whisper/main/assets/thonburian-whisper-logo.png" width="400"/>
---



> By Crews from Looloo Technology and Mahidol University




## **Install Dependencies** ⚙

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!sudo apt install ffmpeg
!pip install torchaudio ipywebrtc notebook
!pip install -q gradio
!pip install pytube
!jupyter nbextension enable --py widgetsnbextension

## **Load and Set-up Thonburian Whisper 🤗**


In [ ]:
import os
import torch
from transformers import pipeline

MODEL_NAME = "biodatlab/whisper-th-medium-combined"
lang = "th"

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

## **Try it with your own voice** 🎥

### Record your own audio here in the notebook!

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
# Save the recorded audio.
recorder.save("audio.mp3")

### Now let our *Thonburian Whisper* do the work!!

In [ ]:
transcriptions = pipe(
    "audio.mp3",
    batch_size=16,
    return_timestamps=False,
    generate_kwargs={"language": "<|th|>", "task": "transcribe"}
)["text"]
print(transcriptions)


## **Transcribe a Youtube Video?**

> [![Watch the video](https://img.youtube.com/vi/jwBqoBIDv3o/default.jpg)](https://www.youtube.com/watch?v=jwBqoBIDv3o)




In [ ]:
import pytube as pt


def yt_transcribe(yt_url: str):
    """Transcribe a given Youtube URL"""
    yt = pt.YouTube(yt_url)
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename="audio.mp3")
    text = pipe(
        "audio.mp3",
        generate_kwargs={"language": "<|th|>", "task": "transcribe"},
        return_timestamps=False,
        batch_size=16
    )
    return text

In [ ]:
# This may take some time depending on the length of the video.
url = "https://www.youtube.com/watch?v=jwBqoBIDv3o"

transcriptions = yt_transcribe(url)
print(transcriptions["text"])